# JSON

`.json` 확장자를 가지는 파일을 로더로 로드하는 방법을 살펴보겠습니다.

- 참고: https://python.langchain.com/docs/modules/data_connection/document_loaders/json


In [1]:
import json
from pathlib import Path
from pprint import pprint


file_path = "data/people.json"
# UTF-8 인코딩을 명시적으로 지정하여 한글 파일 읽기 문제 해결
data = json.loads(Path(file_path).read_text(encoding="utf-8"))

pprint(data)

[{'address': {'city': '서울', 'street': '312번지', 'zipCode': '83795'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['요리', '음악 감상', '사진 촬영'],
  'isMarried': True,
  'name': '박시우',
  'phoneNumbers': ['483-4639-1933', '947-4179-7976']},
 {'address': {'city': '서울', 'street': '877번지', 'zipCode': '36780'},
  'age': 31,
  'carOwnership': True,
  'hobbies': ['여행', '음악 감상', '등산'],
  'isMarried': False,
  'name': '정수아',
  'phoneNumbers': ['337-5721-3227', '387-3768-9586']},
 {'address': {'city': '서울', 'street': '175번지', 'zipCode': '89067'},
  'age': 43,
  'carOwnership': True,
  'hobbies': ['등산', '독서', '게임'],
  'isMarried': False,
  'name': '최도윤',
  'phoneNumbers': ['354-5563-4638', '471-9212-1826']},
 {'address': {'city': '서울', 'street': '690번지', 'zipCode': '70635'},
  'age': 22,
  'carOwnership': False,
  'hobbies': ['여행', '등산', '게임'],
  'isMarried': False,
  'name': '정민준',
  'phoneNumbers': ['468-2796-2152', '922-5760-7030']},
 {'address': {'city': '서울', 'street': '151번지', 'zipCode': '7911

In [ ]:
type(data[0])

## JSONLoader

JSON 데이터의 메시지 키 내 content 필드 아래의 값을 추출하고 싶다고 가정하였을 때, 아래와 같이 JSONLoader를 통해 쉽게 수행할 수 있습니다.

In [4]:
from langchain_community.document_loaders import JSONLoader

# JSONLoader 생성 (encoding 매개변수 제거)
loader = JSONLoader(
    file_path="data/people.json",
    jq_schema=".[].phoneNumbers",
    text_content=False,
)

# 문서 로드
docs = loader.load()

# 결과 출력
pprint(docs)

[Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_num': 1}, page_content='["483-4639-1933", "947-4179-7976"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_num': 2}, page_content='["337-5721-3227", "387-3768-9586"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_num': 3}, page_content='["354-5563-4638", "471-9212-1826"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_num': 4}, page_content='["468-2796-2152", "922-5760-7030"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_num': 5}, page_content='["751-2823-8259", "722-7267-9516"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\github\\langchain-kr\\06-DocumentLoader\\data\\people.json', 'seq_

In [5]:
# 인코딩 문제가 계속 발생하는 경우 대안 방법
import tempfile
import os


# 1. JSON 파일을 UTF-8로 읽어서 임시 파일로 저장
def create_utf8_temp_file(file_path):
    try:
        # UTF-8로 읽기 시도
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()
    except UnicodeDecodeError:
        # 실패하면 다른 인코딩들 시도
        for encoding in ["cp949", "euc-kr", "latin-1"]:
            try:
                with open(file_path, "r", encoding=encoding) as f:
                    content = f.read()
                break
            except UnicodeDecodeError:
                continue
        else:
            raise Exception("모든 인코딩 시도 실패")

    # 임시 파일로 UTF-8로 저장
    temp_file = tempfile.NamedTemporaryFile(
        mode="w", suffix=".json", delete=False, encoding="utf-8"
    )
    temp_file.write(content)
    temp_file.close()

    return temp_file.name


# 2. 임시 파일을 사용하여 JSONLoader 실행
try:
    temp_file_path = create_utf8_temp_file("data/people.json")

    loader = JSONLoader(
        file_path=temp_file_path,
        jq_schema=".[].phoneNumbers",
        text_content=False,
    )

    docs = loader.load()
    pprint(docs)

finally:
    # 임시 파일 정리
    if "temp_file_path" in locals():
        os.unlink(temp_file_path)

[Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 1}, page_content='["483-4639-1933", "947-4179-7976"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 2}, page_content='["337-5721-3227", "387-3768-9586"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 3}, page_content='["354-5563-4638", "471-9212-1826"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 4}, page_content='["468-2796-2152", "922-5760-7030"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 5}, page_content='["751-2823-8259", "722-7267-9516"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\Local\\Temp\\tmpsbdomhn7.json', 'seq_num': 6}, page_content='["462-4433-5968", "483-1709-4850"]'),
 Document(metadata={'source': 'C:\\Users\\park0\\AppData\\